In [1]:
import shelve

with shelve.open('my_shelf') as s:
    s['d'] = {'key': 'value'}

s = shelve.open('my_shelf', 'r')
s['d']

{'key': 'value'}

In [2]:
import dbm
dbm.whichdb('my_shelf')

'dbm.dumb'

In [3]:
import sqlite3
db = sqlite3.connect('cheese_emporium.db')

db.execute('CREATE TABLE cheese(id INTEGER, name TEXT)')
db.executemany(
    'INSERT INTO cheese VALUES (?, ?)',
    [(1, 'red leicester'),
     (2, 'wensleydale'),
     (3, 'cheddar'),
    ]
)
db.commit()
db.close()

In [4]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Date, Integer, String, Table, ForeignKey
from sqlalchemy.orm import relationship

Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    def __repr__(self):
       return "<Customer(name='%s')>" % self.name

purchases_cheeses = Table(
    'purchases_cheeses', Base.metadata,
    Column('purch_id', Integer, ForeignKey('purchases.id', primary_key=True)),
    Column('cheese_id', Integer, ForeignKey('cheeses.id', primary_key=True))
)

class Cheese(Base):
    __tablename__ = 'cheeses'
    id = Column(Integer, primary_key=True)
    kind = Column(String, nullable=False)
    purchases = relationship(
        'Purchase', secondary='purchases_cheeses', back_populates='cheeses'
    )
    def __repr__(self):
       return "<Cheese(kind='%s')>" % self.kind

class Purchase(Base):
    __tablename__ = 'purchases'
    id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customers.id', primary_key=True))
    purchase_date = Column(Date, nullable=False)
    customer = relationship('Customer')
    cheeses = relationship(
        'Cheese', secondary='purchases_cheeses', back_populates='purchases'
    )
    def __repr__(self):
       return ("<Purchase(customer='%s', dt='%s')>" %
                (self.customer.name, self.purchase_date))

/home/af/Dokumenter/Programs/miniconda3/envs/HitchhikersGuide2Python-py38/lib/python3.8/site-packages/sqlalchemy/sql/base.py:299: SAWarning: Can't validate argument 'primary_key'; can't locate any SQLAlchemy dialect named 'primary'
  util.warn(


In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
sess = Session()

leicester = Cheese(kind='Red Leicester')
camembert = Cheese(kind='Camembert')
sess.add_all((camembert, leicester))
cat = Customer(name='Cat')
sess.add(cat)
sess.commit()

import datetime
d = datetime.date(1971, 12, 18)
p = Purchase(purchase_date=d, customer=cat)
p.cheeses.append(camembert)
sess.add(p)
sess.commit()

In [7]:
for row in sess.query(Purchase,Cheese).filter(Purchase.cheeses):
    print(row)

(<Purchase(customer='Cat', dt='1971-12-18')>, <Cheese(kind='Camembert')>)


In [8]:
from sqlalchemy import func
(sess.query(Purchase,Cheese)
    .filter(Purchase.cheeses)
    .from_self(Cheese.kind, func.count(Purchase.id))
    .group_by(Cheese.kind)
).all()

[('Camembert', 1)]

In [9]:
from django.db import models

class Cheese(models.Model):
    type = models.CharField(max_length=30)

class Customer(models.Model):
    name = models.CharField(max_length=50)

class Purchase(models.Model):
    purchase_date = models.DateField()
    customer = models.ForeignKey(Customer)
    cheeses = models.ManyToManyField(Cheese)

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [10]:
leicester = Cheese.objects.create(type='Red Leicester')
camembert = Cheese.objects.create(type='Camembert')
leicester.save()
camembert.save()

doug = Customer.objects.create(name='Douglas')
doug.save()

# Add a time of purchase
import datetime
now = datetime.datetime(1971, 12, 18, 20)
day = datetime.timedelta(1)

p = Purchase(purchase_date=now - 1 * day, customer=doug)
p.save()
p.cheeses.add(camembert, leicester)

AttributeError: type object 'Cheese' has no attribute 'objects'

In [11]:
# Filter for all purchases that happened in the past 7 days:
queryset = Purchase.objects.filter(purchase_date__gt=now - 7 * day)

# Show who bought what cheeses in the query set:
for v in queryset.values('customer__name', 'cheeses__type'):
    print(v)

# Aggregate purchases by cheese type:
from django.db.models import Count
sales_counts = (
    queryset.values('cheeses__type')
    .annotate(total=Count('cheeses'))
    .order_by('cheeses__type')
)
for sc in sales_counts:
    print(sc)

AttributeError: type object 'Purchase' has no attribute 'objects'

In [13]:
import peewee
database = peewee.SqliteDatabase('peewee.db')

class BaseModel(peewee.Model):
    class Meta:
        database = database

class Customer(BaseModel):
    name = peewee.TextField()

class Purchase(BaseModel):
    purchase_date = peewee.DateField()
    customer = peewee.ForeignKeyField(Customer, related_name='purchases')

class Cheese(BaseModel):
    kind = peewee.TextField()

class PurchaseCheese(BaseModel):
    """For the many-to-many relationship."""
    purchase = peewee.ForeignKeyField(Purchase)
    cheese = peewee.ForeignKeyField(Cheese)

database.create_tables((Customer, Purchase, Cheese, PurchaseCheese))

In [14]:
leicester = Cheese.create(kind='Red Leicester')
camembert = Cheese.create(kind='Camembert')
cat = Customer.create(name='Cat')

import datetime
d = datetime.date(1971, 12, 18)

p = Purchase.create(purchase_date=d, customer=cat)
PurchaseCheese.create(purchase=p, cheese=camembert)
PurchaseCheese.create(purchase=p, cheese=leicester)

<PurchaseCheese: 2>

In [15]:
for p in Purchase.select().where(Purchase.purchase_date > d - 1 * day):
    print(p.customer.name, p.purchase_date)

NameError: name 'day' is not defined

In [16]:
from peewee import fn
q = (Cheese
    .select(Cheese.kind, fn.COUNT(Purchase.id).alias('num_purchased'))
    .join(PurchaseCheese)
    .join(Purchase)
    .group_by(Cheese.kind)
 )
for chz in q:
    print(chz.kind, chz.num_purchased)

Camembert 1
Red Leicester 1


In [17]:
import datetime
from pony import orm

db = orm.Database()
db.bind('sqlite', ':memory:')

class Cheese(db.Entity):
    type = orm.Required(str)
    purchases = orm.Set(lambda: Purchase)

class Customer(db.Entity):
    name = orm.Required(str)
    purchases = orm.Set(lambda: Purchase)

class Purchase(db.Entity):
    date = orm.Required(datetime.date)
    customer = orm.Required(Customer)
    cheeses = orm.Set(Cheese)

db.generate_mapping(create_tables=True)

In [18]:
import records
db = records.Database('sqlite:///mydb.db')

rows = db.query('SELECT * FROM cheese')
print(rows.dataset)

OperationalError: (sqlite3.OperationalError) no such table: cheese
[SQL: SELECT * FROM cheese]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [19]:
print(rows.export('json'))

NameError: name 'rows' is not defined